 # MTH8408 : Méthodes d'optimisation et contrôle optimal
 ## Laboratoire 6: Optimisation avec contraintes et contrôle optimal
Tangi Migot et Dominique Orban

Dans ce lab, on va utiliser une branche particulière (en cours de développement) du package PDENLPModels.jl.

In [ ]:
] add https://github.com/JuliaSmoothOptimizers/PDENLPModels.jl#lab_poly_2022

In [ ]:
using PDENLPModels

In [ ]:
using Gridap, NLPModelsIpopt, Plots, Printf, Test

In [ ]:
] status

## Exercice 1: Commande optimale

Dans cet exercice, on considère le problème de gestion de portefeuille vu en cours:
$$
\max_{x,u} \int_0^T (1-u(t))x(t) dt, \quad x(0)=x_0, \dot{x}(t) = \gamma u(t) x(t), \quad 0 \leq u(t) \leq 1
$$
modélisé avec `PDENLPModels`. Compléter l'appel à la fonction `GridapOptimalControl1DNLPModel` avec `n = 100`, `T = 1`, `γ = 3` et `x0 = 0.1`.

In [ ]:
#=function GridapOptimalControl1DNLPModel(f, con, domain, n; 
                                       x0 = nothing, xf = nothing, 
                                       umin = nothing, umax = nothing) =#
# 
# n est la taille de la discrétisation (entier)
n = 
# le domain au format (t₀, T)
domain = 
# x0 est la valeur initiale de x (nombre)
x0 =  
# umin et umax sont des fonctions qui représentent les bornes:
umin = t -> 
umax = t -> 
#La fonction objectif sous l'intégrale:
f(x, u) = 
#Le membre de droite de l'EDO:
γ = 
con(x, u) = 
nlp = GridapOptimalControl1DNLPModel(f, con, domain, n; x0 = x0, umin = umin, umax = umax)

Sur ce type de NLPModel, on peut avoir accès à la taille de la discrétisation de x et de u avec les attributs `nvar_pde` et `nvar_con`. Compléter le test suivant:

In [ ]:
nU = nlp.pdemeta.nvar_pde
nUcon = nlp.pdemeta.nvar_con
@test nU == n
@test nUcon == n+1

### Question 1: Résoudre et affichage
Résoudre le NLPModel avec `ipopt` en partant du vecteur 0. Puis, compléter le test.

In [ ]:
xu0 = # à compléter
stats = ipopt(nlp, x0 = xu0)

In [ ]:
@test :first_order == stats.status

Afficher la solution x et la solution u avec un titre, pas de légende, et le temps en abscisse.

Vous pouvez utiliser la fonction `LinRange` pour gérer le temps en abscisse.

In [ ]:
xsol = stats.solution[1:nU];
usol = stats.solution[nU+1:nU+nUcon];

In [ ]:
# plot x

In [ ]:
# plot u

Pour afficher l'adjoint, compléter la fonction `adjoint_function_final_condition` et afficher le rendu en fonction du temps comme sur les graphes précédents.

In [ ]:
#function adjoint_function_final_condition(domain, n, mul, xf)
# domain et n, cf. l'appel GridapOptimalControlNLPModel
# mul: le multiplicateur de Lagrange renvoyé par Ipopt
# xf: 0

### Question 2: Bang-bang
A quelle instant `t` se situe le changement de trajectoire?

Proposer une nouvelle valeur de γ où le controle n'est plus "bang-bang".

### Question 3: Proposer une modification continue de γ et recommencer les graphiques

## Exercice 2: Le réservoir

Dans ce dernier exercice, on considère le problème de réservoir, où on impose les contraintes $x_1(1)=γ$ et $x_1(t)\leq γ$.

$$
\max_{x_1,x_2,u} x_2(T), \quad \dot{x_1} = -x_1 + u, \dot{x_2} = x_1, x_1(0) = x_2(0) = 0.
$$
Compléter l'appel à la fonction `bounded_2D_optimal_control` avec $n=10$ et $γ = 0.5$.

In [ ]:
#function 2D_bounded_optimal_control(domain, n, f, con1, con2, umin, umax, xmin, xmax, x0, xf)
#n et domain comme dans l'exercice précédent.
n = 
domain = 
γ = 
#le vecteur de donné initial
x0 = zeros(2)
#le nombre pour la contrainte finale sur x_1
xf = γ
#umin, umax, xmin, xmax les fonctions de contraintes de bornes:
umin = t -> 
umax = t -> 
xmin = t -> 
xmax = t -> 
# con1 est le membre de droite de la première EDO, con2 pour la deuxième
con1(x₁, x₂, u) = 
con2(x₁, x₂, u) = 
# f est la fonction objectif sous l'intégrale. Avec PDENLPModels, on ne peux pas ajouter l'instant finale.
# Mais on peut créer une fonction δ(t) = 0 si t < 0.99 et 1 sinon.
δ(t) = t[1] > 0.999 ? 1. : 0. #Dirac of 1
function f(x, u)
  x₁, x₂ = x
  - (δ * x₂)
end
nlp = GridapOptimalControl2DNLPModel(f, con1, con2, domain, n, x0 = x0, xf = xf, umin = umin, umax = umax, xmin = xmin, xmax = xmax)

Résoudre le problème avec Ipopt et découper le résultat dans les vecteurs $x₁, x₂, u$.

In [ ]:
stats = ipopt(nlp, print_level =0, dual_inf_tol = 1e-10)

nn = Integer((nlp.pdemeta.nvar_pde+1)/2)
x₁, x₂, u = stats.solution[1:nn-1], stats.solution[nn:2*nn-1], stats.solution[2*nn:end]

@test length(x₁) + length(x₂) == nlp.pdemeta.nvar_pde
@test length(u) == nlp.pdemeta.nvar_con

Afficher les états et le contrôle.

In [ ]:
# plot x1

In [ ]:
# plot x2

In [ ]:
# plot u

In [ ]:
p₁ = stats.multipliers[1:length(x₁)]
# PDENLPModels.adjoint_function(domain, n, p₁)
# cf. exercices précédents pour le domain et n.
# p₁ est le vecteur de multiplicateur de Lagrange rendu pour la 1ère contrainte
adjoint1 = PDENLPModels.adjoint_function() # à compléter

In [ ]:
p₂ = stats.multipliers[length(x₁)+1:end]
# function adjoint_function_final_condition(domain, n, p₂, 0.0)
# cf. exercice précédent.
adjoint2 = adjoint_function_final_condition() # à compléter

### Question: Convergence en n

Recommencer le processus (avec une boucle) pour observer la convergence du contrôle pour plusieurs valeurs de n (100, 500, 1000).

## Exercice 3: Git/Github, Pull request, OptimizationProblems

L'objectif de cet exercice est de compléter votre implémentation d'un problème d'optimisation et d'ouvrir une "Pull Request" pour l'ajouter au package `OptimizationProblems.jl`.

A tout moment, n'hésitez pas à demander de l'aide sur Zulip sur cette partie.

### Etape 1: Compléter l'implémentation des problèmes

Dans le rapport du Lab2, vous avez choisi un problème d'optimisation à modéliser (merci de le reprendre :) ).

- [ ] Créer une fonction qui crée un modèle JuMP du problème que vous avez choisi. Vous pouvez suivre le modèle de la fonction `arglina` ici https://github.com/JuliaSmoothOptimizers/OptimizationProblems.jl/blob/main/src/PureJuMP/arglina.jl
- [ ] Créer une fonction qui crée un modèle ADNLPModel du problème que vous avez choisi. Vous pouvez suivre le modèle de la fonction `arglina` ici https://github.com/JuliaSmoothOptimizers/OptimizationProblems.jl/blob/main/src/ADNLPProblems/arglina.jl

Quand vous pensez avoir fini, merci de confirmer l'implémentation des deux problèmes avec le professeur.

A noter que les problèmes en version `ADNLPModels` ont un kwargs `type`, car ces problèmes peuvent être formulés sous plusieurs format, cf. exemple ci-dessous. Demandez un coup de main si difficultés de ce côté.

In [ ]:
using ADNLPModels, OptimizationProblems.ADNLPProblems

In [ ]:
nlp = arglina(type = Val(Float32))
obj(nlp, nlp.meta.x0) # renvoie un nombre en Float32

In [ ]:
nlp = arglina()
obj(nlp, nlp.meta.x0) # par défaut on utilise Float64

### Etape 2: Clone & fork

Afin de faire une proposition de modification au package `OptimizationProblems.jl` vous allez devoir "cloner" ce package sur votre compte github et ouvrir une nouvelle branche où ajouter la modification.

- [ ] Aller sur https://github.com/JuliaSmoothOptimizers/OptimizationProblems.jl et cliquer sur "Fork" en haut à droite de l'écran.
- [ ] En suivant les indications du README du lab1 ici https://github.com/tmigot/MTH8408/tree/main/lab1 cloner la version sur votre compte github.
- [ ] En suivant les inditions du README du lab2 ici https://github.com/tmigot/MTH8408/tree/main/lab2 ouvrir une nouvelle branche de travail. (en général on essaye d'éviter de travailler directement sur la branche `main`).

### Etape 3: Modifier le package et mise en ligne
- [ ] Sur votre branch local modifier le dépot en suivant les indications https://juliasmoothoptimizers.github.io/OptimizationProblems.jl/dev/contributing/
- [ ] Une fois les modifications satisfaisantes faire un `git push origin nom_de_votre_branch` pour mettre en ligne vos modifications.
- [ ] Checker vos modifications avec le professeur.
- [ ] Ouvrir la Pull Request!!!